# Optimización de Consultas

creamos la sesión de Spark

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` 

import org.apache.spark.sql.{NotebookSparkSession, SparkSession}

val spark: SparkSession = 
    NotebookSparkSession
      .builder()
      .appName("Queries Optimization")
      .master("local[*]")
      .getOrCreate()


Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/06/11 15:35:27 INFO SparkContext: Running Spark version 2.4.5
21/06/11 15:35:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/06/11 15:35:28 INFO SparkContext: Submitted application: Queries Optimization
21/06/11 15:35:28 INFO SecurityManager: Changing view acls to: jovyan
21/06/11 15:35:28 INFO SecurityManager: Changing modify acls to: jovyan
21/06/11 15:35:28 INFO SecurityManager: Changing view acls groups to: 
21/06/11 15:35:28 INFO SecurityManager: Changing modify acls groups to: 
21/06/11 15:35:28 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
21/06/11 15:35:29 INFO Utils: Successfully started service 'sparkDriver' on port 3

21/06/11 15:35:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0-sources.jar at spark://0a2bcb32361b:34009/jars/scala-collection-compat_2.12-2.0.0-sources.jar with timestamp 1623425729823
21/06/11 15:35:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0.jar at spark://0a2bcb32361b:34009/jars/scala-collection-compat_2.12-2.0.0.jar with timestamp 1623425729824
21/06/11 15:35:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.12/1.2.0/scala-xml_2.12-1.2.0-sources.jar at spark://0a2bcb32361b:34009/jars/scala-xml_2.12-1.2.0-sources.jar with timestamp 1623425729824
21/06/11 15:35:29 INFO SparkContext: Added JAR file:/home/jovyan/.cache

Spark UI

import $ivy.$                                   


import org.apache.spark.sql.{NotebookSparkSession, SparkSession}


spark: SparkSession = org.apache.spark.sql.SparkSession@3f997637

In [2]:
import $ivy.`org.plotly-scala::plotly-almond:0.8.1`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

import $ivy.$                                      


import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

In [3]:
import $ivy.`ch.cern.sparkmeasure:spark-measure_2.12:0.17`

import $ivy.$                                             

Logging

In [4]:
import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}
Logger.getRootLogger().setLevel(Level.ERROR)

import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


imports

In [5]:
import spark.implicits._
import spark.sqlContext.implicits._
import org.apache.spark.sql._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark._
import org.apache.spark.sql.types._, func._
import org.apache.spark.sql.functions.{col, to_date}

import spark.implicits._

import spark.sqlContext.implicits._

import org.apache.spark.sql._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.types._

import org.apache.spark.rdd.RDD

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.types.{IntegerType, StringType, StructType}

import org.apache.spark.{SparkConf, SparkContext}

import org.apache.spark._

import org.apache.spark.sql.types._, func._

import org.apache.spark.sql.functions.{col, to_date}

# Los Datos

El dataset ha sido obtenido de:
https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide

En el se observan los casos diarios de Covid-19 por país hasta el 14-12-20

En la segunda parte se utilizan los datos de las medidas aplicadas a cada país por fecha de inicio y fin:

https://www.ecdc.europa.eu/en/publications-data/download-data-response-measures-covid-19

La consulta para calcular las infecciones por km2:

https://www.kaggle.com/tanuprabhu/population-by-country-2020

Y por último trabajaremos también con vacunaciones:

https://www.kaggle.com/gpreda/covid-world-vaccination-progress

tendremos que utilizar otro dataset con datos de covid ya que en el original no vienen todas las fechas:

https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset

## Creo una clase para trabajar con infecciones 

In [6]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Infection(day : Int, 
                     month : Int, 
                     year : Int, 
                     nCases: Int, 
                     nDeaths : Int, 
                     country : String,  
                     continent : String) 
extends Serializable

defined class Infection

Y un método para medir tiempos de ejecución

In [7]:
def runWithOutput[A](code: => A): Int = {
    val start = System.currentTimeMillis()
    val res = code
    val out = System.currentTimeMillis() - start
    println(s"Took ${System.currentTimeMillis() - start}")
    out.toInt
}

defined function runWithOutput

### Para utilizar showHTML()

In [8]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit:Int = 20, truncate: Int = 20) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map { cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map { row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)        
    }
}

defined class RichDF

# Empiezo trabajando con RDDs

In [9]:
val infectionData = spark.sparkContext.textFile("data.csv")

infectionData: RDD[String] = data.csv MapPartitionsRDD[1] at textFile at cmd8.sc:1

Creo una funcion para trabajar con un RDD de infecciones

In [10]:
def infections(lines : RDD[String]) : RDD[Infection] =
    lines.map(line => {
      val arr = line.split(",")
      Infection(
        day = arr(1).toInt,
        month = arr(2).toInt,
        year = arr(3).toInt,
        nCases = arr(4).toInt,
        nDeaths = arr(5).toInt,
        country = arr(6),
        continent = arr(10)
      )
    })

defined function infections

Calculo la media de infecciones diarias por país trabajando con pair RDD

In [11]:
  def infectionGrowthAverage(infections : RDD[Infection]) : RDD[(String, Int)]= {

    val countriesAndCases : RDD[(String, Iterable[Int])] = 
      infections.map(x => (x.country,x.nCases))
      .groupByKey()
      
    countriesAndCases.mapValues(x => (x.sum / x.size)).sortBy(_._2)
  }

defined function infectionGrowthAverage

Muestro el resultado y el tiempo de ejecución

In [12]:
val infectionRDD = infections(infectionData)
val infectionAvgRDD = infectionGrowthAverage(infectionRDD)

map at cmd10.sc:4

2 / 2

sortBy at cmd10.sc:7

2 / 2

infectionRDD: RDD[Infection] = MapPartitionsRDD[2] at map at cmd9.sc:2
infectionAvgRDD: RDD[(String, Int)] = MapPartitionsRDD[10] at sortBy at cmd10.sc:7

Usando la API de spark

In [13]:
val timeRDD = spark.time(infectionAvgRDD.collect())

sortBy at cmd10.sc:7

2 / 2

collect at cmd12.sc:1

2 / 2

Time taken: 345 ms


timeRDD: Array[(String, Int)] = Array(
  ("Greenland", 0),
  ("British_Virgin_Islands", 0),
  ("\"Bonaire", 0),
  ("Fiji", 0),
  ("Vanuatu", 0),
  ("Saint_Vincent_and_the_Grenadines", 0),
  ("Solomon_Islands", 0),
  ("Holy_See", 0),
  ("Dominica", 0),
  ("Wallis_and_Futuna", 0),
  ("Grenada", 0),
  ("Marshall_Islands", 0),
  ("Anguilla", 0),
  ("Timor_Leste", 0),
  ("Saint_Kitts_and_Nevis", 0),
  ("Falkland_Islands_(Malvinas)", 0),
  ("Montserrat", 0),
  ("Northern_Mariana_Islands", 0),
  ("Seychelles", 0),
  ("Antigua_and_Barbuda", 0),
  ("Laos", 0),
  ("Brunei_Darussalam", 0),
  ("New_Caledonia", 0),
  ("Guernsey", 1),
  ("Saint_Lucia", 1),
  ("Cayman_Islands", 1),
  ("Bermuda", 1),
  ("Isle_of_Man", 1),
  ("Barbados", 1),
  ("Cambodia", 1),
  ("Faroe_Islands", 1),
  ("Bhutan", 1),
  ("Monaco", 1),
  ("Mauritius", 1),
  ("United_Republic_of_Tanzania", 1),
  ("Papua_New_Guinea", 2),
  ("Eritrea", 2),
  ("Taiwan", 2),
...

o bien el framework del cern que nos da más información

In [14]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(infectionAvgRDD.collect())

collect at cmd13.sc:1

2 / 2

Time taken: 36 ms


take at stagemetrics.scala:207

1 / 1

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 1
numTasks => 2
elapsedTime => 24 (24 ms)
stageDuration => 24 (24 ms)
executorRunTime => 18 (18 ms)
executorCpuTime => 12 (12 ms)
executorDeserializeTime => 7 (7 ms)
executorDeserializeCpuTime => 6 (6 ms)
resultSerializationTime => 1 (1 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 0 (0 ms)
resultSize => 8350 (8.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 30148
recordsRead => 0
bytesRead => 0 (0 Bytes)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 214
shuffleTotalBlocksFetched => 4
shuffleLocalBlocksFetched => 4
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 5413 (5.0 KB)
shuffleLocalBytesRead => 5413 (5.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 0 (0 Bytes)
shuffleRecordsWritten => 0


res13: Array[(String, Int)] = Array(
  ("Greenland", 0),
  ("British_Virgin_Islands", 0),
  ("\"Bonaire", 0),
  ("Fiji", 0),
  ("Vanuatu", 0),
  ("Saint_Vincent_and_the_Grenadines", 0),
  ("Solomon_Islands", 0),
  ("Holy_See", 0),
  ("Dominica", 0),
  ("Wallis_and_Futuna", 0),
  ("Grenada", 0),
  ("Marshall_Islands", 0),
  ("Anguilla", 0),
  ("Timor_Leste", 0),
  ("Saint_Kitts_and_Nevis", 0),
  ("Falkland_Islands_(Malvinas)", 0),
  ("Montserrat", 0),
  ("Northern_Mariana_Islands", 0),
  ("Seychelles", 0),
  ("Antigua_and_Barbuda", 0),
  ("Laos", 0),
  ("Brunei_Darussalam", 0),
  ("New_Caledonia", 0),
  ("Guernsey", 1),
  ("Saint_Lucia", 1),
  ("Cayman_Islands", 1),
  ("Bermuda", 1),
  ("Isle_of_Man", 1),
  ("Barbados", 1),
  ("Cambodia", 1),
  ("Faroe_Islands", 1),
  ("Bhutan", 1),
  ("Monaco", 1),
  ("Mauritius", 1),
  ("United_Republic_of_Tanzania", 1),
  ("Papua_New_Guinea", 2),
  ("Eritrea", 2),
  ("Taiwan", 2),
...

# Hago los mismos calculos con un DataFrame

Convierto el RDD obtenido previamente en un DataFrame para inferir la clase infección

In [15]:
val infectionDF = spark.createDataFrame(infectionRDD)

infectionDF: DataFrame = [day: int, month: int ... 5 more fields]

Utilizo los métodos de la clase DF que incluye uno optimizado para calcular la media.

Ejecuto y comprabamos como el tiempo de ejecución es significativamente menor que en RDD

In [16]:
val infAvgOrDf = infectionDF.
    groupBy("country")
    .avg("nCases")
    .orderBy(desc("avg(nCases)"))

infAvgOrDf: Dataset[Row] = [country: string, avg(nCases): double]

In [17]:
infAvgOrDf.showHTML()

take at cmd7.sc:4

2 / 2

take at cmd7.sc:4

200 / 200

country,avg(nCases)
United_States_of_...,46447.868571428575
India,28321.203438395416
Brazil,19719.862857142856
Russia,7582.651428571428
France,6791.005714285714
Argentina,5293.851590106007
United_Kingdom,5284.0085714285715
Italy,5267.748571428571
Colombia,5110.301075268817
Spain,4958.667621776504


In [18]:
spark.time(infAvgOrDf.count())

count at cmd17.sc:1

2 / 2

count at cmd17.sc:1

200 / 200

count at cmd17.sc:1

...

200 / 200

count at cmd17.sc:1

1 / 1

Time taken: 6127 ms


res17: Long = 214L

In [19]:
val timeDF = spark.time(infAvgOrDf.collect)

collect at cmd18.sc:1

2 / 2

collect at cmd18.sc:1

200 / 200

collect at cmd18.sc:1

200 / 200

collect at cmd18.sc:1

200 / 200

Time taken: 2913 ms


timeDF: Array[Row] = Array(
  [United_States_of_America,46447.868571428575],
  [India,28321.203438395416],
  [Brazil,19719.862857142856],
  [Russia,7582.651428571428],
  [France,6791.005714285714],
  [Argentina,5293.851590106007],
  [United_Kingdom,5284.0085714285715],
  [Italy,5267.748571428571],
  [Colombia,5110.301075268817],
  [Spain,4958.667621776504],
  [Poland,3998.8591549295775],
  [Germany,3820.2228571428573],
  [Turkey,3606.7789855072465],
  [Mexico,3571.554285714286],
  [Peru,3492.8120567375886],
  [Ukraine,3263.282608695652],
  [Iran,3166.482857142857],
  [South_Africa,3053.063829787234],
  [Chile,2006.7333333333333],
  [Indonesia,1801.2244897959183],
  [Bangladesh,1783.7563636363636],
  [Netherlands,1750.702857142857],
  [Belgium,1737.1457142857143],
  [Czechia,1660.2257142857143],
  [Iraq,1651.2471264367816],
  [Romania,1598.6637931034484],
  [Morocco,1427.175],
  [Canada,1316.4085714285713],
  [Philippines,1298.8439306358382],
  [Pakistan,1277.6434782608696],
  [Saudi_Ar

In [20]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(infAvgOrDf.collect)

collect at cmd19.sc:1

200 / 200

Time taken: 659 ms


take at stagemetrics.scala:207

1 / 1

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 1
numTasks => 200
elapsedTime => 621 (0.6 s)
stageDuration => 621 (0.6 s)
executorRunTime => 1334 (1 s)
executorCpuTime => 526 (0.5 s)
executorDeserializeTime => 1700 (2 s)
executorDeserializeCpuTime => 665 (0.7 s)
resultSerializationTime => 46 (46 ms)
jvmGCTime => 77 (77 ms)
shuffleFetchWaitTime => 6 (6 ms)
shuffleWriteTime => 0 (0 ms)
resultSize => 628320 (613.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 432537600
recordsRead => 0
bytesRead => 0 (0 Bytes)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 214
shuffleTotalBlocksFetched => 214
shuffleLocalBlocksFetched => 214
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 17080 (16.0 KB)
shuffleLocalBytesRead => 17080 (16.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 0 (0 Bytes)


res19: Array[Row] = Array(
  [United_States_of_America,46447.868571428575],
  [India,28321.203438395416],
  [Brazil,19719.862857142856],
  [Russia,7582.651428571428],
  [France,6791.005714285714],
  [Argentina,5293.851590106007],
  [United_Kingdom,5284.0085714285715],
  [Italy,5267.748571428571],
  [Colombia,5110.301075268817],
  [Spain,4958.667621776504],
  [Poland,3998.8591549295775],
  [Germany,3820.2228571428573],
  [Turkey,3606.7789855072465],
  [Mexico,3571.554285714286],
  [Peru,3492.8120567375886],
  [Ukraine,3263.282608695652],
  [Iran,3166.482857142857],
  [South_Africa,3053.063829787234],
  [Chile,2006.7333333333333],
  [Indonesia,1801.2244897959183],
  [Bangladesh,1783.7563636363636],
  [Netherlands,1750.702857142857],
  [Belgium,1737.1457142857143],
  [Czechia,1660.2257142857143],
  [Iraq,1651.2471264367816],
  [Romania,1598.6637931034484],
  [Morocco,1427.175],
  [Canada,1316.4085714285713],
  [Philippines,1298.8439306358382],
  [Pakistan,1277.6434782608696],
  [Saudi_Ara

Otra opción es crear el DataFrame directamente importando los datos pero deja de ser un DF de infecciones

In [21]:
val dfCovid = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("covidworldwide.csv")

csv at cmd20.sc:6

1 / 1

csv at cmd20.sc:6

2 / 2

dfCovid: DataFrame = [dateRep: string, day: int ... 10 more fields]

In [22]:
dfCovid.schema

res21: StructType = StructType(
  StructField("dateRep", StringType, true, {}),
  StructField("day", IntegerType, true, {}),
  StructField("month", IntegerType, true, {}),
  StructField("year", IntegerType, true, {}),
  StructField("cases", IntegerType, true, {}),
  StructField("deaths", IntegerType, true, {}),
  StructField("countriesAndTerritories", StringType, true, {}),
  StructField("geoId", StringType, true, {}),
  StructField("countryterritoryCode", StringType, true, {}),
  StructField("popData2019", IntegerType, true, {}),
  StructField("continentExp", StringType, true, {}),
  StructField(
    "Cumulative_number_for_14_days_of_COVID-19_cases_per_100000",
    DoubleType,
    true,
    {}
  )
)

In [23]:
dfCovid.explain

== Physical Plan ==
*(1) FileScan csv [dateRep#569,day#570,month#571,year#572,cases#573,deaths#574,countriesAndTerritories#575,geoId#576,countryterritoryCode#577,popData2019#578,continentExp#579,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000#580] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/jovyan/work/covidworldwide.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<dateRep:string,day:int,month:int,year:int,cases:int,deaths:int,countriesAndTerritories:str...


In [24]:
val dfCovidWithSchema = dfCovid.toDF
    .groupBy("countriesAndTerritories")
    .agg(mean("cases"))
    .orderBy("avg(cases)")

dfCovidWithSchema: Dataset[Row] = [countriesAndTerritories: string, avg(cases): double]

In [25]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(dfCovidWithSchema.collect)

collect at cmd24.sc:1

2 / 2

collect at cmd24.sc:1

200 / 200

collect at cmd24.sc:1

...

4 / 4

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 4
numTasks => 602
elapsedTime => 2817 (3 s)
stageDuration => 2772 (3 s)
executorRunTime => 7572 (8 s)
executorCpuTime => 3190 (3 s)
executorDeserializeTime => 5175 (5 s)
executorDeserializeCpuTime => 2328 (2 s)
resultSerializationTime => 36 (36 ms)
jvmGCTime => 252 (0.3 s)
shuffleFetchWaitTime => 10 (10 ms)
shuffleWriteTime => 3027 (3 s)
resultSize => 676633 (660.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 432537600
recordsRead => 61900
bytesRead => 4372838 (4.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 644
shuffleTotalBlocksFetched => 480
shuffleLocalBlocksFetched => 480
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 43422 (42.0 KB)
shuffleLocalBytesRead => 43422 (42.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 30

res24: Array[Row] = Array(
  [Vanuatu,0.029411764705882353],
  [Anguilla,0.03802281368821293],
  [Montserrat,0.048507462686567165],
  [Wallis_and_Futuna,0.05084745762711865],
  [Greenland,0.07037037037037037],
  [Falkland_Islands_(Malvinas),0.07450980392156863],
  [Marshall_Islands,0.0851063829787234],
  [Holy_See,0.09454545454545454],
  [Saint_Kitts_and_Nevis,0.10227272727272728],
  [Timor_Leste,0.11567164179104478],
  [New_Caledonia,0.13382899628252787],
  [Laos,0.15471698113207547],
  [Fiji,0.17037037037037037],
  [Grenada,0.25842696629213485],
  [Solomon_Islands,0.2833333333333333],
  [British_Virgin_Islands,0.2889733840304182],
  [Dominica,0.3258426966292135],
  [Saint_Vincent_and_the_Grenadines,0.3684210526315789],
  [Northern_Mariana_Islands,0.4362934362934363],
  [Brunei_Darussalam,0.5448028673835126],
  [Antigua_and_Barbuda,0.5481481481481482],
  [Bonaire, Saint Eustatius and Saba,0.6731517509727627],
  [Seychelles,0.68],
  [Saint_Lucia,1.0],
  [Cambodia,1.0527859237536656],
 

puedo definir el esquema manualmente para crear el DataFrame

In [26]:
//Defino el esquema manualmente pero podría verlo importando el csv y viendo como lo hace de base spark

val schema = new StructType()
    .add("dateRep",StringType,true)
    .add("day",IntegerType,true)
    .add("month",IntegerType,true)
    .add("year",IntegerType,true)
    .add("cases",IntegerType,true)
    .add("deaths",IntegerType,true)
    .add("countriesAndTerritories",StringType,true)
    .add("geoId",StringType,true)
    .add("countryterritoryCode",StringType,true)
    .add("popData2018",IntegerType,true)
    .add("continentExp",StringType,true)

schema: StructType = StructType(
  StructField("dateRep", StringType, true, {}),
  StructField("day", IntegerType, true, {}),
  StructField("month", IntegerType, true, {}),
  StructField("year", IntegerType, true, {}),
  StructField("cases", IntegerType, true, {}),
  StructField("deaths", IntegerType, true, {}),
  StructField("countriesAndTerritories", StringType, true, {}),
  StructField("geoId", StringType, true, {}),
  StructField("countryterritoryCode", StringType, true, {}),
  StructField("popData2018", IntegerType, true, {}),
  StructField("continentExp", StringType, true, {})
)

In [27]:
val df = spark.read
.format("csv")
.option("header","true")
.schema(schema)
.load("data.csv")

df: DataFrame = [dateRep: string, day: int ... 9 more fields]

In [28]:
df.printSchema()

root
 |-- dateRep: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- cases: integer (nullable = true)
 |-- deaths: integer (nullable = true)
 |-- countriesAndTerritories: string (nullable = true)
 |-- geoId: string (nullable = true)
 |-- countryterritoryCode: string (nullable = true)
 |-- popData2018: integer (nullable = true)
 |-- continentExp: string (nullable = true)



# Y con un DataSet

In [29]:
val infectionDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.csv("covidworldwide.csv")
.as[(String,String,String,String,String,String,String,String,String,String,String,String)]

csv at cmd28.sc:5

1 / 1

infectionDS: Dataset[(String, String, String, String, String, String, String, String, String, String, String, String)] = [dateRep: string, day: string ... 10 more fields]

In [30]:
val avgDS = 
    infectionDS.groupBy($"countriesAndTerritories")
    .agg(avg($"cases"))
    .orderBy("avg(cases)")
    .as[(String,Double)]

avgDS: Dataset[(String, Double)] = [countriesAndTerritories: string, avg(cases): double]

In [31]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(avgDS.collect)

collect at cmd30.sc:1

2 / 2

collect at cmd30.sc:1

200 / 200

collect at cmd30.sc:1

...

3 / 3

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 3
numTasks => 402
elapsedTime => 2133 (2 s)
stageDuration => 2100 (2 s)
executorRunTime => 6066 (6 s)
executorCpuTime => 2468 (2 s)
executorDeserializeTime => 3556 (4 s)
executorDeserializeCpuTime => 1549 (2 s)
resultSerializationTime => 37 (37 ms)
jvmGCTime => 211 (0.2 s)
shuffleFetchWaitTime => 7 (7 ms)
shuffleWriteTime => 3225 (3 s)
resultSize => 622473 (607.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 325058560
recordsRead => 61900
bytesRead => 4372838 (4.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 430
shuffleTotalBlocksFetched => 266
shuffleLocalBlocksFetched => 266
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 26310 (25.0 KB)
shuffleLocalBytesRead => 26310 (25.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 3026

res30: Array[(String, Double)] = Array(
  ("Vanuatu", 0.029411764705882353),
  ("Anguilla", 0.03802281368821293),
  ("Montserrat", 0.048507462686567165),
  ("Wallis_and_Futuna", 0.05084745762711865),
  ("Greenland", 0.07037037037037037),
  ("Falkland_Islands_(Malvinas)", 0.07450980392156863),
  ("Marshall_Islands", 0.0851063829787234),
  ("Holy_See", 0.09454545454545454),
  ("Saint_Kitts_and_Nevis", 0.10227272727272728),
  ("Timor_Leste", 0.11567164179104478),
  ("New_Caledonia", 0.13382899628252787),
  ("Laos", 0.15471698113207547),
  ("Fiji", 0.17037037037037037),
  ("Grenada", 0.25842696629213485),
  ("Solomon_Islands", 0.2833333333333333),
  ("British_Virgin_Islands", 0.2889733840304182),
  ("Dominica", 0.3258426966292135),
  ("Saint_Vincent_and_the_Grenadines", 0.3684210526315789),
  ("Northern_Mariana_Islands", 0.4362934362934363),
  ("Brunei_Darussalam", 0.5448028673835126),
  ("Antigua_and_Barbuda", 0.5481481481481482),
  ("Bonaire, Saint Eustatius and Saba", 0.6731517509727627

### Trabajamos con Dataset[Infection]

In [32]:
val infectionDataset = spark.createDataset(infectionRDD)

infectionDataset: Dataset[Infection] = [day: int, month: int ... 5 more fields]

In [33]:
val avgInfectionDS = infectionDataset
    .groupBy($"country")
    .agg(avg($"nCases").as[Double])
    .orderBy("avg(nCases)")
    .as[(String,Double)]

avgInfectionDS: Dataset[(String, Double)] = [country: string, avg(nCases): double]

In [34]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(avgInfectionDS.collect)

collect at cmd33.sc:1

2 / 2

collect at cmd33.sc:1

200 / 200

collect at cmd33.sc:1

...

3 / 3

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 3
numTasks => 402
elapsedTime => 2889 (3 s)
stageDuration => 2848 (3 s)
executorRunTime => 9901 (10 s)
executorCpuTime => 3822 (4 s)
executorDeserializeTime => 4514 (5 s)
executorDeserializeCpuTime => 2206 (2 s)
resultSerializationTime => 26 (26 ms)
jvmGCTime => 264 (0.3 s)
shuffleFetchWaitTime => 31 (31 ms)
shuffleWriteTime => 5849 (6 s)
resultSize => 573544 (560.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 325058560
recordsRead => 61900
bytesRead => 4350839 (4.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 430
shuffleTotalBlocksFetched => 328
shuffleLocalBlocksFetched => 328
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 29370 (28.0 KB)
shuffleLocalBytesRead => 29370 (28.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 3

res33: Array[(String, Double)] = Array(
  ("Vanuatu", 0.029411764705882353),
  ("Anguilla", 0.03802281368821293),
  ("Montserrat", 0.048507462686567165),
  ("Wallis_and_Futuna", 0.05084745762711865),
  ("Greenland", 0.07037037037037037),
  ("Falkland_Islands_(Malvinas)", 0.07450980392156863),
  ("Marshall_Islands", 0.0851063829787234),
  ("Holy_See", 0.09454545454545454),
  ("Saint_Kitts_and_Nevis", 0.10227272727272728),
  ("Timor_Leste", 0.11567164179104478),
  ("New_Caledonia", 0.13382899628252787),
  ("Laos", 0.15471698113207547),
  ("Fiji", 0.17037037037037037),
  ("Grenada", 0.25842696629213485),
  ("Solomon_Islands", 0.2833333333333333),
  ("British_Virgin_Islands", 0.2889733840304182),
  ("Dominica", 0.3258426966292135),
  ("Saint_Vincent_and_the_Grenadines", 0.3684210526315789),
  ("Northern_Mariana_Islands", 0.4362934362934363),
  ("Brunei_Darussalam", 0.5448028673835126),
  ("Antigua_and_Barbuda", 0.5481481481481482),
  ("\"Bonaire", 0.6731517509727627),
  ("Seychelles", 0.68

# Utilizo una segunda tabla y cruzo datos con RDD, DS y DF

## Creo una consulta para calcular la media de infecciones por Km2

### Utilizando RDDs

In [35]:
val populationData = spark.sparkContext.textFile("population_by_country_2020.csv")

populationData: RDD[String] = population_by_country_2020.csv MapPartitionsRDD[144] at textFile at cmd34.sc:1

In [36]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Population(
    country : String, 
    population : Int, 
    density : Int, 
    land_area: Int, 
    ) 
extends Serializable

defined class Population

Limpio la primera linea del CSV y creo un RDD de población

In [37]:
val header = populationData.first() 

def population(lines : RDD[String]) : RDD[Population] =
    lines.filter(x => x != header)
    .map(line => {
      val arr = line.split(",")
      Population(
        country = arr(0),
        population = arr(1).toInt,
        density = arr(4).toInt,
        land_area = arr(5).toInt,
      )
    })

first at cmd36.sc:1

1 / 1

header: String = "Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km\u00b2),Land Area (Km\u00b2),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share"
defined function population

Compruebo que se visualizan correctamente los datos

In [38]:
val populationRDD = population(populationData)

populationRDD: RDD[Population] = MapPartitionsRDD[146] at map at cmd36.sc:5

In [39]:
populationRDD.toDF.showHTML()

take at cmd7.sc:4

1 / 1

country,population,density,land_area
China,1440297825,153,9388211
India,1382345085,464,2973190
United States,331341050,36,9147420
Indonesia,274021604,151,1811570
Pakistan,221612785,287,770880
Brazil,212821986,25,8358140
Nigeria,206984347,226,910770
Bangladesh,164972348,1265,130170
Russia,145945524,9,16376870
Mexico,129166028,66,1943950


### Un join computacionalmente pesado desde el principio ya que cruza todos los datos sin quedarnos con los que nos interesen

Spark no me deja hacer un Join de RDD que no sean pair RDD así que tenemos que construirlo

In [39]:
// populationRDD.join(infectionRDD)

Construyo Pair RDDs conservando todos los datos

In [40]:
val populationByCountry = populationRDD.map(
    x => (x.country,x))

val infectionByCountry = 
      infectionRDD.map(x => (x.country,x))

populationByCountry: RDD[(String, Population)] = MapPartitionsRDD[151] at map at cmd39.sc:1
infectionByCountry: RDD[(String, Infection)] = MapPartitionsRDD[152] at map at cmd39.sc:5

Hago el Join y agrupo por paises

In [41]:
val megaRDD = infectionByCountry.join(populationByCountry).groupByKey()

megaRDD: RDD[(String, Iterable[(Infection, Population)])] = MapPartitionsRDD[156] at groupByKey at cmd40.sc:1

Finalmente calculo la media

In [42]:
megaRDD.mapValues(
    x => x.map( 
        line => line._1.nCases.toFloat / line._2.land_area.toFloat
    )).mapValues(
    x => x.sum / x.size
).collect()

map at cmd39.sc:5

2 / 2

map at cmd39.sc:1

2 / 2

collect at cmd41.sc:5

2 / 2

res41: Array[(String, Float)] = Array(
  ("Australia", 1.04250785E-5F),
  ("Cura\u00e7ao", 0.030684844F),
  ("Brazil", 0.0023593607F),
  ("Canada", 1.4476357E-4F),
  ("Uzbekistan", 6.44254E-4F),
  ("Barbados", 0.0024965815F),
  ("Japan", 0.0014080021F),
  ("Yemen", 1.5844575E-5F),
  ("Greenland", 1.714469E-7F),
  ("Cyprus", 0.0058788075F),
  ("Finland", 2.930214E-4F),
  ("Peru", 0.0027287588F),
  ("Grenada", 7.600793E-4F),
  ("Singapore", 0.23804064F),
  ("Benin", 1.0037852E-4F),
  ("Djibouti", 9.118434E-4F),
  ("Montenegro", 0.011323532F),
  ("Lesotho", 3.463115E-4F),
  ("Uruguay", 2.0170162E-4F),
  ("Moldova", 0.013754949F),
  ("Gibraltar", 0.39814794F),
  ("Colombia", 0.0046059503F),
  ("Sweden", 0.0022288002F),
  ("Vietnam", 1.3021491E-5F),
  ("Dominica", 4.3445695E-4F),
  ("Serbia", 0.010879752F),
  ("Iran", 0.0019443579F),
  ("Madagascar", 1.127008E-4F),
  ("Slovakia", 0.009806483F),
  ("Angola", 4.8450314E-5F),
  ("Taiwan", 5.9708724E-5F),
  ("Denmark", 0.00739087F),
  ("Luxembo

Lo hago todo en una única operación para calcular el tiempo de ejecución

In [43]:
val notOptimizedRDD =
    infectionByCountry.join(populationByCountry)
    .groupByKey()
    .mapValues(
    x => x.map( 
        line => line._1.nCases.toFloat / line._2.land_area.toFloat)
    ).mapValues(
        x => x.sum / x.size
    )

notOptimizedRDD: RDD[(String, Float)] = MapPartitionsRDD[164] at mapValues at cmd42.sc:8

¿Hay alguna diferencia cruzando los datos en orden inverso? Parece que no

In [44]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    populationByCountry.join(infectionByCountry)
    .groupByKey()
    .mapValues(
    x => x.map( 
        line => line._1.land_area.toFloat / line._2.nCases.toFloat)
    ).mapValues(
        x => x.sum / x.size
    ).collect()
)

map at cmd39.sc:1

2 / 2

map at cmd39.sc:5

2 / 2

collect at cmd43.sc:8

...

3 / 3

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 3
numTasks => 6
elapsedTime => 950 (1.0 s)
stageDuration => 964 (1.0 s)
executorRunTime => 1780 (2 s)
executorCpuTime => 1639 (2 s)
executorDeserializeTime => 37 (37 ms)
executorDeserializeCpuTime => 20 (20 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 88 (88 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 11 (11 ms)
resultSize => 6631 (6.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 17046837
recordsRead => 62136
bytesRead => 4374674 (4.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 62135
shuffleTotalBlocksFetched => 8
shuffleLocalBlocksFetched => 8
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 1009263 (985.0 KB)
shuffleLocalBytesRead => 1009263 (985.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 1009263 

res43: Array[(String, Float)] = Array(
  ("Australia", InfinityF),
  ("Jordan", InfinityF),
  ("Cura\u00e7ao", InfinityF),
  ("Brazil", InfinityF),
  ("Canada", InfinityF),
  ("Uzbekistan", InfinityF),
  ("Barbados", InfinityF),
  ("Japan", InfinityF),
  ("Yemen", InfinityF),
  ("Greenland", InfinityF),
  ("Cyprus", InfinityF),
  ("Finland", InfinityF),
  ("Peru", InfinityF),
  ("Singapore", InfinityF),
  ("Benin", InfinityF),
  ("Djibouti", InfinityF),
  ("Montenegro", InfinityF),
  ("Lesotho", InfinityF),
  ("Uruguay", InfinityF),
  ("Moldova", InfinityF),
  ("Gibraltar", InfinityF),
  ("Colombia", InfinityF),
  ("Sweden", InfinityF),
  ("Vietnam", InfinityF),
  ("Dominica", InfinityF),
  ("Serbia", InfinityF),
  ("Iran", InfinityF),
  ("Madagascar", InfinityF),
  ("Slovakia", InfinityF),
  ("Angola", InfinityF),
  ("Taiwan", InfinityF),
  ("Denmark", InfinityF),
  ("Luxembourg", InfinityF),
  ("Cameroon", InfinityF),
  ("Turkey", InfinityF),
  ("Mozambique", InfinityF),
  ("Congo", 

#### Para optimizar un poco esta consulta:

Despejo solo los datos que me interesan para trabajar con Pair RDDs y optimizar la consulta

In [45]:
val countriesAndLandArea = populationRDD.map(
    x => (x.country,x.land_area))

countriesAndLandArea: RDD[(String, Int)] = MapPartitionsRDD[179] at map at cmd44.sc:1

In [46]:
val countriesAndCases = 
      infectionRDD.map(x => (x.country,x.nCases))
      .groupByKey()

countriesAndCases: RDD[(String, Iterable[Int])] = ShuffledRDD[181] at groupByKey at cmd45.sc:2

Ejecuto un join y trabajo para calcular primero la media de infecciones por Km2 diaria, 
para luego calcular la media total

In [47]:
val average = countriesAndCases.join(countriesAndLandArea)

average: RDD[(String, (Iterable[Int], Int))] = MapPartitionsRDD[184] at join at cmd46.sc:1

In [48]:
average.mapValues(
    x => x._1.map(
        y => (y.toFloat / x._2.toFloat)
    )).mapValues(
    x => x.sum/x.size
).collect()

map at cmd45.sc:2

2 / 2

map at cmd44.sc:1

2 / 2

collect at cmd47.sc:5

2 / 2

res47: Array[(String, Float)] = Array(
  ("Australia", 1.04250785E-5F),
  ("Brazil", 0.0023593607F),
  ("Uzbekistan", 6.44254E-4F),
  ("Yemen", 1.5844575E-5F),
  ("Greenland", 1.714469E-7F),
  ("Finland", 2.930214E-4F),
  ("Peru", 0.0027287588F),
  ("Djibouti", 9.118434E-4F),
  ("Montenegro", 0.011323532F),
  ("Lesotho", 3.463115E-4F),
  ("Uruguay", 2.0170162E-4F),
  ("Gibraltar", 0.39814794F),
  ("Sweden", 0.0022288002F),
  ("Vietnam", 1.3021491E-5F),
  ("Serbia", 0.010879752F),
  ("Iran", 0.0019443579F),
  ("Madagascar", 1.127008E-4F),
  ("Angola", 4.8450314E-5F),
  ("Slovakia", 0.009806483F),
  ("Denmark", 0.00739087F),
  ("Paraguay", 8.4728445E-4F),
  ("Eswatini", 0.0014308676F),
  ("Suriname", 1.2708931E-4F),
  ("Iceland", 1.5837558E-4F),
  ("Netherlands", 0.05191881F),
  ("Mongolia", 2.142477E-6F),
  ("Kazakhstan", 2.5090596E-4F),
  ("Fiji", 9.325142E-6F),
  ("Indonesia", 9.94289E-4F),
  ("Eritrea", 2.6267177E-5F),
  ("Somalia", 2.6736525E-5F),
  ("Norway", 3.13054E-4F),
  ("Libe

Lo hago todo en una única operación para calcular el tiempo de ejecución

In [49]:
val meanInfectionsRDD =
countriesAndCases.join(countriesAndLandArea)   
.mapValues(
    x => x._1.map(
        y => (y.toDouble / x._2.toDouble)
    )).mapValues(
    x => x.sum / x.size
)

meanInfectionsRDD: RDD[(String, Double)] = MapPartitionsRDD[191] at mapValues at cmd48.sc:7

In [50]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    meanInfectionsRDD.collect
)

map at cmd44.sc:1

2 / 2

collect at cmd49.sc:2

2 / 2

Time taken: 295 ms


...

2 / 2

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 2
numTasks => 4
elapsedTime => 287 (0.3 s)
stageDuration => 287 (0.3 s)
executorRunTime => 484 (0.5 s)
executorCpuTime => 136 (0.1 s)
executorDeserializeTime => 36 (36 ms)
executorDeserializeCpuTime => 19 (19 ms)
resultSerializationTime => 2 (2 ms)
jvmGCTime => 32 (32 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 6 (6 ms)
resultSize => 7310 (7.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 922460
recordsRead => 236
bytesRead => 23835 (23.0 KB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 62135
shuffleTotalBlocksFetched => 8
shuffleLocalBlocksFetched => 8
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 202188 (197.0 KB)
shuffleLocalBytesRead => 202188 (197.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 4210 (4.0 KB)
s

res49: Array[(String, Double)] = Array(
  ("Australia", 1.0425077311296284E-5),
  ("Brazil", 0.0023593601994155214),
  ("Uzbekistan", 6.442540983325267E-4),
  ("Yemen", 1.5844578001381826E-5),
  ("Greenland", 1.7144687628303176E-7),
  ("Finland", 2.930212997287386E-4),
  ("Peru", 0.0027287594193262437),
  ("Djibouti", 9.118434583828152E-4),
  ("Montenegro", 0.0113235294117647),
  ("Lesotho", 3.463115511063499E-4),
  ("Uruguay", 2.017016237105369E-4),
  ("Gibraltar", 0.39814814814814803),
  ("Sweden", 0.002228799810610017),
  ("Vietnam", 1.3021486291273772E-5),
  ("Serbia", 0.01087974910979714),
  ("Iran", 0.0019443571625942454),
  ("Madagascar", 1.1270081941955635E-4),
  ("Angola", 4.845029684336793E-5),
  ("Slovakia", 0.009806489520984575),
  ("Denmark", 0.007390862260529942),
  ("Paraguay", 8.472839146251595E-4),
  ("Eswatini", 0.001430866807610994),
  ("Suriname", 1.270892687559355E-4),
  ("Iceland", 1.5837548984681146E-4),
  ("Netherlands", 0.05191882731740392),
  ("Mongolia", 2.14

## Consulta con DataSet

In [51]:
val infectionDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("covidworldwide.csv")
.withColumnRenamed("countriesAndTerritories","Country")
.as[(String,String,String,String,Double,Double,String,String,String,String,String,String)]

csv at cmd50.sc:6

1 / 1

csv at cmd50.sc:6

2 / 2

infectionDS: Dataset[(String, String, String, String, Double, Double, String, String, String, String, String, String)] = [dateRep: string, day: int ... 10 more fields]

In [52]:
val populationDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("population_by_country_2020.csv")
.withColumnRenamed("Country (or dependency)","Country")
.withColumnRenamed("Population (2020)","Population")
.as[(String,Float,String,Float,Float,Float,Double,String,String,String,String)]

csv at cmd51.sc:6

1 / 1

csv at cmd51.sc:6

1 / 1

populationDS: Dataset[(String, Float, String, Float, Float, Float, Double, String, String, String, String)] = [Country: string, Population: int ... 9 more fields]

### infecciones por Km2

In [53]:
val meanInfectionsperKM2DS = 
infectionDS.join(populationDS, "Country")
        .select($"Country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
                $"cases" / $"Land Area (Km\u00b2)" as "infection Per Km\u00b2")
        .groupBy("Country")
        .agg(round(avg("infection Per Km\u00b2"),10).as[Float])
        .orderBy(desc("round(avg(infection Per Km²), 10)"))
        .as[(String,Double)]

meanInfectionsperKM2DS: Dataset[(String, Double)] = [Country: string, round(avg(infection Per Km²), 10): double]

In [54]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(meanInfectionsperKM2DS.collect)

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd53.sc:1

2 / 2

collect at cmd53.sc:1

...

4 / 4

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 4
numTasks => 403
elapsedTime => 2654 (3 s)
stageDuration => 2447 (2 s)
executorRunTime => 5843 (6 s)
executorCpuTime => 2482 (2 s)
executorDeserializeTime => 4907 (5 s)
executorDeserializeCpuTime => 2410 (2 s)
resultSerializationTime => 32 (32 ms)
jvmGCTime => 195 (0.2 s)
shuffleFetchWaitTime => 3 (3 ms)
shuffleWriteTime => 2235 (2 s)
resultSize => 785250 (766.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 283115520
recordsRead => 62135
bytesRead => 4388728 (4.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 316
shuffleTotalBlocksFetched => 224
shuffleLocalBlocksFetched => 224
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 21232 (20.0 KB)
shuffleLocalBytesRead => 21232 (20.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 2278

res53: Array[(String, Double)] = Array(
  ("Monaco", 1.9763313609),
  ("Gibraltar", 0.3981481481),
  ("Bahrain", 0.336084301),
  ("Singapore", 0.2380408163),
  ("Maldives", 0.1591428571),
  ("Malta", 0.1225817138),
  ("Aruba", 0.1050561798),
  ("Belgium", 0.0573694093),
  ("Andorra", 0.0565679926),
  ("Netherlands", 0.0519188273),
  ("Guam", 0.0485581522),
  ("Israel", 0.0476568368),
  ("Luxembourg", 0.0464581199),
  ("Lebanon", 0.0413947417),
  ("Qatar", 0.0350905886),
  ("Liechtenstein", 0.0339157706),
  ("Bermuda", 0.0319259259),
  ("Cura\u00e7ao", 0.0306848202),
  ("Switzerland", 0.0269206687),
  ("Kuwait", 0.0236463256),
  ("Italy", 0.0179089841),
  ("Slovenia", 0.0168388184),
  ("Armenia", 0.015314984),
  ("Moldova", 0.0137549467),
  ("Bangladesh", 0.0137032831),
  ("Portugal", 0.013267124),
  ("Poland", 0.0130583521),
  ("France", 0.0124023722),
  ("Montenegro", 0.0113235294),
  ("Austria", 0.0110957368),
  ("Hungary", 0.0110410058),
  ("Germany", 0.0109600151),
  ("Serbia", 0.0

### infecciones por número de habitantes

In [55]:
val meanInfectionPerPopulationDS = 
infectionDS.join(populationDS, "Country")
        .select($"Country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
               $"cases" / $"Population" as "infection Per Population")
        .groupBy("country")
        .avg("infection Per Population")
        .orderBy(desc("avg(infection Per Population)"))
        .as[(String,Double)]

meanInfectionPerPopulationDS: Dataset[(String, Double)] = [country: string, avg(infection Per Population): double]

In [56]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(meanInfectionPerPopulationDS.collect)

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd55.sc:1

2 / 2

collect at cmd55.sc:1

...

4 / 4

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 4
numTasks => 403
elapsedTime => 1851 (2 s)
stageDuration => 1762 (2 s)
executorRunTime => 4584 (5 s)
executorCpuTime => 1935 (2 s)
executorDeserializeTime => 3532 (4 s)
executorDeserializeCpuTime => 1650 (2 s)
resultSerializationTime => 23 (23 ms)
jvmGCTime => 239 (0.2 s)
shuffleFetchWaitTime => 2 (2 ms)
shuffleWriteTime => 1900 (2 s)
resultSize => 785207 (766.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 283115520
recordsRead => 62135
bytesRead => 4388728 (4.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 316
shuffleTotalBlocksFetched => 224
shuffleLocalBlocksFetched => 224
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 21238 (20.0 KB)
shuffleLocalBytesRead => 21238 (20.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 2278

res55: Array[(String, Double)] = Array(
  ("Andorra", 3.4400295679401656E-4),
  ("Montenegro", 2.4248737515640602E-4),
  ("Luxembourg", 1.9169310791749766E-4),
  ("Aruba", 1.769864042262208E-4),
  ("Slovenia", 1.6312438537043818E-4),
  ("Panama", 1.5933045595717767E-4),
  ("Guam", 1.5512777072857744E-4),
  ("Belgium", 1.4977394087242467E-4),
  ("Bahrain", 1.4927852711388843E-4),
  ("Armenia", 1.470935836076611E-4),
  ("Liechtenstein", 1.4225295036283446E-4),
  ("Qatar", 1.410043921276158E-4),
  ("Georgia", 1.3807441257860555E-4),
  ("Croatia", 1.2360663598057857E-4),
  ("Switzerland", 1.2276072071712674E-4),
  ("Portugal", 1.1923139268591238E-4),
  ("Israel", 1.1883296974895626E-4),
  ("Gibraltar", 1.1818342727541584E-4),
  ("Argentina", 1.1694609939486568E-4),
  ("Moldova", 1.1205780134087437E-4),
  ("Serbia", 1.0898339588681578E-4),
  ("Spain", 1.0604965444992492E-4),
  ("Peru", 1.0568198964713385E-4),
  ("Poland", 1.056801767088061E-4),
  ("Chile", 1.0481978256189323E-4),
  ("France

## Consulta con DataFrame

In [57]:
val dfCovid = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("covidworldwide.csv")

csv at cmd56.sc:6

1 / 1

csv at cmd56.sc:6

2 / 2

dfCovid: DataFrame = [dateRep: string, day: int ... 10 more fields]

In [58]:
val dfMeasures = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("response_graphs_data_2021-04-15.csv")
dfMeasures.show
dfMeasures.schema

csv at cmd57.sc:6

1 / 1

csv at cmd57.sc:6

1 / 1

show at cmd57.sc:7

1 / 1

+-------+--------------------+-------------------+----------+
|Country|    Response_measure|         date_start|  date_end|
+-------+--------------------+-------------------+----------+
|Austria|AdaptationOfWorkp...|2020-03-16 00:00:00|2020-04-13|
|Austria|AdaptationOfWorkp...|2021-04-01 00:00:00|        NA|
|Austria|      BanOnAllEvents|2020-11-03 00:00:00|        NA|
|Austria|         ClosDaycare|2020-03-16 00:00:00|2020-05-04|
|Austria|  ClosDaycarePartial|2020-11-17 00:00:00|2020-12-07|
|Austria|            ClosHigh|2020-03-16 00:00:00|2020-09-30|
|Austria|     ClosHighPartial|2020-11-03 00:00:00|        NA|
|Austria|            ClosPrim|2020-03-16 00:00:00|2020-05-18|
|Austria|            ClosPrim|2020-12-26 00:00:00|2021-02-07|
|Austria|     ClosPrimPartial|2020-11-17 00:00:00|2020-12-07|
|Austria|          ClosPubAny|2020-03-16 00:00:00|2020-04-13|
|Austria|          ClosPubAny|2020-05-02 00:00:00|2020-05-14|
|Austria|          ClosPubAny|2020-11-16 00:00:00|2020-12-06|
|Austria

dfMeasures: DataFrame = [Country: string, Response_measure: string ... 2 more fields]
res57_2: StructType = StructType(
  StructField("Country", StringType, true, {}),
  StructField("Response_measure", StringType, true, {}),
  StructField("date_start", TimestampType, true, {}),
  StructField("date_end", StringType, true, {})
)

In [59]:
val dfPopulation = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("population_by_country_2020.csv")
.withColumnRenamed("Country (or dependency)","Country")
.withColumnRenamed("Population (2020)","Population")
dfPopulation.showHTML()
dfPopulation.schema

csv at cmd58.sc:6

1 / 1

csv at cmd58.sc:6

1 / 1

take at cmd7.sc:4

1 / 1

Country,Population,Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
China,1440297825,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
India,1382345085,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
United States,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
Indonesia,274021604,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %
Brazil,212821986,0.72 %,1509890,25,8358140,21200.0,1.7,33,88 %,2.73 %
Nigeria,206984347,2.58 %,5175990,226,910770,-60000.0,5.4,18,52 %,2.64 %
Bangladesh,164972348,1.01 %,1643222,1265,130170,-369501.0,2.1,28,39 %,2.11 %
Russia,145945524,0.04 %,62206,9,16376870,182456.0,1.8,40,74 %,1.87 %
Mexico,129166028,1.06 %,1357224,66,1943950,-60000.0,2.1,29,84 %,1.65 %


dfPopulation: DataFrame = [Country: string, Population: int ... 9 more fields]
res58_2: StructType = StructType(
  StructField("Country", StringType, true, {}),
  StructField("Population", IntegerType, true, {}),
  StructField("Yearly Change", StringType, true, {}),
  StructField("Net Change", IntegerType, true, {}),
  StructField("Density (P/Km\u00b2)", IntegerType, true, {}),
  StructField("Land Area (Km\u00b2)", IntegerType, true, {}),
  StructField("Migrants (net)", DoubleType, true, {}),
  StructField("Fert. Rate", StringType, true, {}),
  StructField("Med. Age", StringType, true, {}),
  StructField("Urban Pop %", StringType, true, {}),
  StructField("World Share", StringType, true, {})
)

Modifico los datos de entrada para que el formato fecha se adecue al TimeStamp de Spark

In [60]:
val dfCovidClean = dfCovid
    .select($"*",$"dateRep",translate($"dateRep","/","-").as("date"))
    .drop("dateRep")

dfCovidClean: DataFrame = [day: int, month: int ... 10 more fields]

In [61]:
val dfCovidDate = dfCovidClean
    .select($"*",col("date"),to_date(col("date"),"dd-MM-yyyy").as("to_date"))

dfCovidDate: DataFrame = [day: int, month: int ... 12 more fields]

Hago una consulta de prueba para obtener la media solo de los casos en España

In [62]:
val spainCovid = dfCovid.select("dateRep","cases").where("countriesAndTerritories == 'Spain'").toDF

spainCovid: DataFrame = [dateRep: string, cases: int]

In [63]:
spainCovid.agg(avg("cases")).showHTML()

take at cmd7.sc:4

2 / 2

take at cmd7.sc:4

1 / 1

avg(cases)
4958.667621776504


Cruzo los datos con un Join y hago algunas consultas sencillas

In [64]:
val megaDF = dfCovid.join(dfMeasures, $"Country" === $"countriesAndTerritories")

megaDF: DataFrame = [dateRep: string, day: int ... 14 more fields]

In [65]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    
    dfCovid.join(dfMeasures, $"Country" === $"countriesAndTerritories")
        .select("cases","deaths","dateRep","Response_measure")
        .where("countriesAndTerritories == 'Spain'")
        .collect()
)

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd64.sc:5

2 / 2

Time taken: 449 ms


...

2 / 2

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 2
numTasks => 3
elapsedTime => 319 (0.3 s)
stageDuration => 274 (0.3 s)
executorRunTime => 319 (0.3 s)
executorCpuTime => 207 (0.2 s)
executorDeserializeTime => 8 (8 ms)
executorDeserializeCpuTime => 6 (6 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 13 (13 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 0 (0 ms)
resultSize => 205930 (201.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 4198400
recordsRead => 63509
bytesRead => 4455274 (4.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 0
shuffleTotalBlocksFetched => 0
shuffleLocalBlocksFetched => 0
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 0 (0 Bytes)
shuffleLocalBytesRead => 0 (0 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 0 (0 Bytes)
shuffleRecordsW

res64: Array[Row] = Array(
  [0,0,13/12/2020,TeleworkingPartial],
  [0,0,13/12/2020,Teleworking],
  [0,0,13/12/2020,StayHomeRiskG],
  [0,0,13/12/2020,StayHomeOrderPartial],
  [0,0,13/12/2020,StayHomeOrderPartial],
  [0,0,13/12/2020,StayHomeOrder],
  [0,0,13/12/2020,StayHomeGen],
  [0,0,13/12/2020,SocialCircle],
  [0,0,13/12/2020,RestaurantsCafesPartial],
  [0,0,13/12/2020,RestaurantsCafes],
  [0,0,13/12/2020,PrivateGatheringRestrictions],
  [0,0,13/12/2020,PlaceOfWorshipPartial],
  [0,0,13/12/2020,OutdoorOver1000],
  [0,0,13/12/2020,MassGatherAllPartial],
  [0,0,13/12/2020,MassGatherAll],
  [0,0,13/12/2020,MassGatherAll],
  [0,0,13/12/2020,MassGather50Partial],
  [0,0,13/12/2020,MassGather50],
  [0,0,13/12/2020,MassGather50],
  [0,0,13/12/2020,MasksVoluntaryClosedSpaces],
  [0,0,13/12/2020,MasksMandatoryClosedSpaces],
  [0,0,13/12/2020,MasksMandatoryClosedSpaces],
  [0,0,13/12/2020,MasksMandatoryAllSpacesPartial],
  [0,0,13/12/2020,MasksMandatoryAllSpaces],
  [0,0,13/12/2020,IndoorOver

### Finalmente ejecuto la consulta de nuestro caso de uso, infecciones por Km2

In [66]:
val meanInfectionsperKM2DF = 
dfCovid.join(dfPopulation, $"country" === $"countriesAndTerritories")
        .select($"country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
                $"cases" / $"Land Area (Km\u00b2)" as "infection Per Km\u00b2")
        .groupBy("country")
        .avg("infection Per Km\u00b2")
        .orderBy(desc("avg(infection Per Km²)"))

meanInfectionsperKM2DF: Dataset[Row] = [country: string, avg(infection Per Km²): double]

In [67]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    meanInfectionsperKM2DF.collect
    )

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd66.sc:2

2 / 2

collect at cmd66.sc:2

...

4 / 4


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 4
numTasks => 403
elapsedTime => 3044 (3 s)
stageDuration => 2911 (3 s)
executorRunTime => 4759 (5 s)
executorCpuTime => 1983 (2 s)
executorDeserializeTime => 4041 (4 s)
executorDeserializeCpuTime => 1709 (2 s)
resultSerializationTime => 21 (21 ms)
jvmGCTime => 335 (0.3 s)
shuffleFetchWaitTime => 11 (11 ms)
shuffleWriteTime => 2256 (2 s)
resultSize => 784949 (766.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 283115520
recordsRead => 62135
bytesRead => 4388728 (4.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 316
shuffleTotalBlocksFetched => 224
shuffleLocalBlocksFetched => 224
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 21232 (20.0 KB)
shuffleLocalBytesRead => 21232 (20.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 22

take at stagemetrics.scala:207

1 / 1

res66: Array[Row] = Array(
  [Monaco,1.9763313609467457],
  [Gibraltar,0.39814814814814803],
  [Bahrain,0.3360843010104055],
  [Singapore,0.23804081632653054],
  [Maldives,0.15914285714285714],
  [Malta,0.12258171378091859],
  [Aruba,0.10505617977528105],
  [Belgium,0.05736940932251372],
  [Andorra,0.05656799259944492],
  [Netherlands,0.05191882731740392],
  [Guam,0.048558152248189136],
  [Israel,0.047656836789593385],
  [Luxembourg,0.04645811992750769],
  [Lebanon,0.04139474174902102],
  [Qatar,0.03509058863945274],
  [Liechtenstein,0.03391577060931893],
  [Bermuda,0.03192592592592586],
  [Curaçao,0.030684820158504333],
  [Switzerland,0.026920668662241683],
  [Kuwait,0.023646325567555148],
  [Italy,0.01790898406006858],
  [Slovenia,0.016838818411961356],
  [Armenia,0.015314984029080356],
  [Moldova,0.013754946727549473],
  [Bangladesh,0.013703283119277588],
  [Portugal,0.013267124014649443],
  [Poland,0.013058352071742075],
  [France,0.012402372199215262],
  [Montenegro,0.011323529411

### Media de casos por número de habitante

In [68]:
val infectionsPerPopulation = dfCovid.join(dfPopulation, $"country" === $"countriesAndTerritories")
        .select($"country",
                $"dateRep" as "date",
                $"cases",
                $"Population",
                $"cases" / $"Population" as "infection Per Population")
        .groupBy("country")
        .avg("infection Per Population")
        .orderBy(desc("avg(infection Per Population)"))

infectionsPerPopulation: Dataset[Row] = [country: string, avg(infection Per Population): double]

### Porcentaje diario de infectados

In [69]:
val diaryInfectionsDF =
dfCovidDate.join(dfPopulation, $"country" === $"countriesAndTerritories")
        .select($"country",
                $"to_date",
                $"day",
                $"month",
                $"cases",
                $"Population",
                $"cases" / $"Population" as "infection Per Population")
        .orderBy($"to_date".asc)

diaryInfectionsDF: Dataset[Row] = [country: string, to_date: date ... 5 more fields]

In [70]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    diaryInfectionsDF.collect
    )

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd69.sc:2

2 / 2

collect at cmd69.sc:2

...

3 / 3

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 3
numTasks => 5
elapsedTime => 1333 (1 s)
stageDuration => 1122 (1 s)
executorRunTime => 1122 (1 s)
executorCpuTime => 993 (1.0 s)
executorDeserializeTime => 66 (66 ms)
executorDeserializeCpuTime => 29 (29 ms)
resultSerializationTime => 18 (18 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 123 (0.1 s)
resultSize => 2857726 (2.0 MB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 4210688
recordsRead => 124035
bytesRead => 8761566 (8.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 0
shuffleTotalBlocksFetched => 0
shuffleLocalBlocksFetched => 0
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 0 (0 Bytes)
shuffleLocalBytesRead => 0 (0 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 1344022 (1312.0 KB)
shu

res69: Array[Row] = Array(
  [Afghanistan,2019-12-31,31,12,0,39074280,0.0],
  [Algeria,2019-12-31,31,12,0,43984569,0.0],
  [Armenia,2019-12-31,31,12,0,2964219,0.0],
  [Australia,2019-12-31,31,12,0,25550683,0.0],
  [Austria,2019-12-31,31,12,0,9015361,0.0],
  [Azerbaijan,2019-12-31,31,12,0,10154978,0.0],
  [Bahrain,2019-12-31,31,12,0,1711057,0.0],
  [Belarus,2019-12-31,31,12,0,9448772,0.0],
  [Belgium,2019-12-31,31,12,0,11598451,0.0],
  [Brazil,2019-12-31,31,12,0,212821986,0.0],
  [Cambodia,2019-12-31,31,12,0,16758448,0.0],
  [Canada,2019-12-31,31,12,0,37799407,0.0],
  [China,2019-12-31,31,12,27,1440297825,1.874612287219138E-8],
  [Croatia,2019-12-31,31,12,0,4100719,0.0],
  [Denmark,2019-12-31,31,12,0,5795780,0.0],
  [Ecuador,2019-12-31,31,12,0,17688599,0.0],
  [Egypt,2019-12-31,31,12,0,102659126,0.0],
  [Estonia,2019-12-31,31,12,0,1326693,0.0],
  [Finland,2019-12-31,31,12,0,5542237,0.0],
  [France,2019-12-31,31,12,0,65298930,0.0],
  [Georgia,2019-12-31,31,12,0,3987805,0.0],
  [Germany,2

# Consulta con vacunaciones

## Comparativa de infecciones frente a vacunaciones

In [71]:
val dfCovid2 = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("covid_19_data.csv")
dfCovid2.schema

csv at cmd70.sc:6

1 / 1

csv at cmd70.sc:6

5 / 5

dfCovid2: DataFrame = [SNo: int, ObservationDate: string ... 6 more fields]
res70_1: StructType = StructType(
  StructField("SNo", IntegerType, true, {}),
  StructField("ObservationDate", StringType, true, {}),
  StructField("Province/State", StringType, true, {}),
  StructField("Country/Region", StringType, true, {}),
  StructField("Last Update", StringType, true, {}),
  StructField("Confirmed", DoubleType, true, {}),
  StructField("Deaths", DoubleType, true, {}),
  StructField("Recovered", DoubleType, true, {})
)

In [72]:
val vaccinations = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("country_vaccinations.csv")
vaccinations.schema

csv at cmd71.sc:6

1 / 1

csv at cmd71.sc:6

1 / 1

vaccinations: DataFrame = [country: string, iso_code: string ... 13 more fields]
res71_1: StructType = StructType(
  StructField("country", StringType, true, {}),
  StructField("iso_code", StringType, true, {}),
  StructField("date", TimestampType, true, {}),
  StructField("total_vaccinations", DoubleType, true, {}),
  StructField("people_vaccinated", DoubleType, true, {}),
  StructField("people_fully_vaccinated", DoubleType, true, {}),
  StructField("daily_vaccinations_raw", DoubleType, true, {}),
  StructField("daily_vaccinations", DoubleType, true, {}),
  StructField("total_vaccinations_per_hundred", DoubleType, true, {}),
  StructField("people_vaccinated_per_hundred", DoubleType, true, {}),
  StructField("people_fully_vaccinated_per_hundred", DoubleType, true, {}),
  StructField("daily_vaccinations_per_million", DoubleType, true, {}),
  StructField("vaccines", StringType, true, {}),
  StructField("source_name", StringType, true, {}),
  StructField("source_website", StringType, true

modifico los datos de entrada para que se ajuste la fecha

In [73]:
val vaccinationsClean = vaccinations
    .select($"*",col("date"),to_date(col("date"),"MM-dd-yyyy")
            .as("dateVaccinated"))
    .drop("date")

vaccinationsClean: DataFrame = [country: string, iso_code: string ... 13 more fields]

In [74]:
val dfCovidClean2 = dfCovid2
    .select($"*",$"ObservationDate",translate($"ObservationDate","/","-")
            .as("date1"))
    .drop("ObservationDate")
    .select($"*",col("date1"),to_date(col("date1"),"MM-dd-yyyy")
            .as("date"))
    .drop("date1")

dfCovidClean2: DataFrame = [SNo: int, Province/State: string ... 6 more fields]

triple join

In [75]:
dfCovidClean2.join(
    vaccinationsClean,$"date" === $"dateVaccinated"
    && dfCovidClean2("Country/Region") <=> vaccinationsClean("country")
).join(dfPopulation, "country").showHTML()

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

take at cmd7.sc:4

1 / 1

take at cmd7.sc:4

4 / 4

country,SNo,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,date,iso_code,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website,dateVaccinated,Population,Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
Norway,169377,null,Norway,2021-04-02 15:13:53,36969.0,351.0,17998.0,2020-12-02,NOR,1.0,1.0,null,null,null,0.0,0.0,null,null,"Moderna, Pfizer/B...",Norwegian Institu...,https://www.fhi.n...,2020-12-02,5428594,0.79 %,42384,15,365268,28000.0,1.7,40,83 %,0.07 %
Norway,170139,null,Norway,2021-04-02 15:13:53,37371.0,353.0,17998.0,2020-12-03,NOR,1.0,1.0,null,0.0,0.0,0.0,0.0,null,0.0,"Moderna, Pfizer/B...",Norwegian Institu...,https://www.fhi.n...,2020-12-03,5428594,0.79 %,42384,15,365268,28000.0,1.7,40,83 %,0.07 %
Norway,170901,null,Norway,2021-04-02 15:13:53,37801.0,354.0,17998.0,2020-12-04,NOR,2.0,2.0,null,1.0,0.0,0.0,0.0,null,0.0,"Moderna, Pfizer/B...",Norwegian Institu...,https://www.fhi.n...,2020-12-04,5428594,0.79 %,42384,15,365268,28000.0,1.7,40,83 %,0.07 %
Norway,171663,null,Norway,2021-04-02 15:13:53,38073.0,354.0,17998.0,2020-12-05,NOR,2.0,2.0,null,0.0,0.0,0.0,0.0,null,0.0,"Moderna, Pfizer/B...",Norwegian Institu...,https://www.fhi.n...,2020-12-05,5428594,0.79 %,42384,15,365268,28000.0,1.7,40,83 %,0.07 %
Norway,172425,null,Norway,2021-04-02 15:13:53,38323.0,354.0,17998.0,2020-12-06,NOR,2.0,2.0,null,0.0,0.0,0.0,0.0,null,0.0,"Moderna, Pfizer/B...",Norwegian Institu...,https://www.fhi.n...,2020-12-06,5428594,0.79 %,42384,15,365268,28000.0,1.7,40,83 %,0.07 %
Norway,173187,null,Norway,2021-04-02 15:13:53,38703.0,359.0,17998.0,2020-12-07,NOR,4.0,4.0,null,2.0,1.0,0.0,0.0,null,0.0,"Moderna, Pfizer/B...",Norwegian Institu...,https://www.fhi.n...,2020-12-07,5428594,0.79 %,42384,15,365268,28000.0,1.7,40,83 %,0.07 %
Norway,173949,null,Norway,2021-04-02 15:13:53,39163.0,361.0,17998.0,2020-12-08,NOR,4.0,4.0,null,0.0,0.0,0.0,0.0,null,0.0,"Moderna, Pfizer/B...",Norwegian Institu...,https://www.fhi.n...,2020-12-08,5428594,0.79 %,42384,15,365268,28000.0,1.7,40,83 %,0.07 %
Norway,174711,null,Norway,2021-04-02 15:13:53,39525.0,361.0,17998.0,2020-12-09,NOR,5.0,5.0,null,1.0,1.0,0.0,0.0,null,0.0,"Moderna, Pfizer/B...",Norwegian Institu...,https://www.fhi.n...,2020-12-09,5428594,0.79 %,42384,15,365268,28000.0,1.7,40,83 %,0.07 %
Norway,175473,null,Norway,2021-04-02 15:13:53,40023.0,382.0,17998.0,2020-12-10,NOR,6.0,6.0,null,1.0,1.0,0.0,0.0,null,0.0,"Moderna, Pfizer/B...",Norwegian Institu...,https://www.fhi.n...,2020-12-10,5428594,0.79 %,42384,15,365268,28000.0,1.7,40,83 %,0.07 %
Norway,176235,null,Norway,2021-04-02 15:13:53,40439.0,387.0,17998.0,2020-12-11,NOR,7.0,7.0,null,1.0,1.0,0.0,0.0,null,0.0,"Moderna, Pfizer/B...",Norwegian Institu...,https://www.fhi.n...,2020-12-11,5428594,0.79 %,42384,15,365268,28000.0,1.7,40,83 %,0.07 %


In [76]:
val megaQuerie = dfCovidClean2.join(
    vaccinationsClean,$"date" === $"dateVaccinated"
    && dfCovidClean2("Country/Region") <=> vaccinationsClean("country")
).join(dfPopulation,"country")
        .select($"country",
                $"date",
                $"confirmed",
                $"people_vaccinated",
                $"Population",
                $"confirmed" / $"Population" as "infection Per Population",
                $"people_vaccinated"/ $"Population" as "vaccination Per Population",
                $"people_vaccinated" / $"confirmed" as "infection-vaccination rate")
        .orderBy($"date".asc)
        .na.fill("")
        .withColumn("infection-vaccination rate", round($"infection-vaccination rate",8))
        .withColumn("vaccination Per Population", round($"vaccination Per Population",8))

megaQuerie: DataFrame = [country: string, date: date ... 6 more fields]

In [77]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    megaQuerie.collect
    )

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd76.sc:2

...

6 / 6

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 6
numTasks => 142
elapsedTime => 2961 (3 s)
stageDuration => 2704 (3 s)
executorRunTime => 8395 (8 s)
executorCpuTime => 5755 (6 s)
executorDeserializeTime => 1527 (2 s)
executorDeserializeCpuTime => 670 (0.7 s)
resultSerializationTime => 66 (66 ms)
jvmGCTime => 456 (0.5 s)
shuffleFetchWaitTime => 4 (4 ms)
shuffleWriteTime => 73 (73 ms)
resultSize => 2732572 (2.0 MB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 276889600
recordsRead => 647465
bytesRead => 50729325 (48.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 64698
shuffleTotalBlocksFetched => 130
shuffleLocalBlocksFetched => 130
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 2224463 (2.0 MB)
shuffleLocalBytesRead => 2224463 (2.0 MB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten 

res76: Array[Row] = Array(
  [Norway,2020-12-02,36969.0,1.0,5428594,0.006810050631894741,1.8E-7,2.705E-5],
  [Norway,2020-12-03,37371.0,1.0,5428594,0.0068841029555719215,1.8E-7,2.676E-5],
  [Norway,2020-12-04,37801.0,2.0,5428594,0.00696331315254005,3.7E-7,5.291E-5],
  [Norway,2020-12-05,38073.0,2.0,5428594,0.0070134182073664005,3.7E-7,5.253E-5],
  [Norway,2020-12-06,38323.0,2.0,5428594,0.00705947064746415,3.7E-7,5.219E-5],
  [Norway,2020-12-07,38703.0,4.0,5428594,0.007129470356412729,7.4E-7,1.0335E-4],
  [Norway,2020-12-08,39163.0,4.0,5428594,0.007214206846192587,7.4E-7,1.0214E-4],
  [Norway,2020-12-09,39525.0,5.0,5428594,0.007280890779454128,9.2E-7,1.265E-4],
  [Norway,2020-12-10,40023.0,6.0,5428594,0.0073726272401288435,1.11E-6,1.4991E-4],
  [Norway,2020-12-11,40439.0,7.0,5428594,0.007449258500451498,1.29E-6,1.731E-4],
  [Norway,2020-12-12,40803.0,7.0,5428594,0.00751631085323382,1.29E-6,1.7156E-4],
  [Norway,2020-12-13,41003.0,7.0,5428594,0.00755315280531202,1.29E-6,1.7072E-4],
  [No

# Consulta utilizando los datos en .parquet

In [77]:
/*dfCovid.write
    .partitionBy("countriesAndTerritories","cases")
    .parquet("data_files/covid.parquet")
*/

In [78]:
val parqDF = spark.read.parquet("data_files/covid.parquet")

parquet at cmd77.sc:1

214 / 214

parquet at cmd77.sc:1

1 / 1

parqDF: DataFrame = [dateRep: string, day: int ... 10 more fields]

casos por km2

In [79]:
val parquetCasesKM2 =
parqDF.join(dfPopulation, $"country" === $"countriesAndTerritories")
        .select($"country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
                $"cases" / $"Land Area (Km\u00b2)" as "infection Per Km\u00b2")
        .groupBy("country")
        .avg("infection Per Km\u00b2")
        .orderBy(desc("avg(infection Per Km²)"))

parquetCasesKM2: Dataset[Row] = [country: string, avg(infection Per Km²): double]

In [80]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    parquetCasesKM2
    .collect()
)

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd79.sc:2

863 / 863

collect at cmd79.sc:2

...

200 / 200

collect at cmd79.sc:2

157 / 157

Time taken: 152883 ms

Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 4
numTasks => 1264
elapsedTime => 152094 (2.5 min)
stageDuration => 151736 (2.5 min)
executorRunTime => 892556 (15 min)
executorCpuTime => 107681 (1.8 min)
executorDeserializeTime => 12203 (12 s)
executorDeserializeCpuTime => 7719 (8 s)
resultSerializationTime => 60 (60 ms)
jvmGCTime => 8378 (8 s)
shuffleFetchWaitTime => 918 (0.9 s)
shuffleWriteTime => 16731 (17 s)
resultSize => 2725104 (2.0 MB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 2043142144
recordsRead => 62135
bytesRead => 124565546 (118.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 33032
shuffleTotalBlocksFetched => 31664
shuffleLocalBlocksFetched => 31664
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 2621982 (2.0 MB)
shuffleLocalBytesRead => 2621982 (2.0 MB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffle

res79: Array[Row] = Array(
  [Monaco,1.9763313609467457],
  [Gibraltar,0.3981481481481482],
  [Bahrain,0.33608430101040576],
  [Singapore,0.23804081632653057],
  [Maldives,0.15914285714285717],
  [Malta,0.12258171378091871],
  [Aruba,0.10505617977528085],
  [Belgium,0.05736940932251368],
  [Andorra,0.05656799259944495],
  [Netherlands,0.051918827317403854],
  [Guam,0.04855815224818914],
  [Israel,0.047656836789593426],
  [Luxembourg,0.04645811992750771],
  [Lebanon,0.041394741749021066],
  [Qatar,0.03509058863945274],
  [Liechtenstein,0.03391577060931899],
  [Bermuda,0.03192592592592593],
  [Curaçao,0.030684820158504374],
  [Switzerland,0.026920668662241708],
  [Kuwait,0.023646325567555162],
  [Italy,0.017908984060068567],
  [Slovenia,0.01683881841196134],
  [Armenia,0.015314984029080353],
  [Moldova,0.01375494672754946],
  [Bangladesh,0.013703283119277583],
  [Portugal,0.013267124014649443],
  [Poland,0.013058352071742083],
  [France,0.012402372199215262],
  [Montenegro,0.011323529411

casos por número de habitante

In [81]:
val parquetCasesPopulation =
parqDF.join(dfPopulation, $"country" === $"countriesAndTerritories")
        .select($"country",
                $"dateRep" as "date",
                $"cases",
                $"Population",
                $"cases" / $"Population" as "infection Per Population")
        .groupBy("country")
        .avg("infection Per Population")
        .orderBy(desc("avg(infection Per Population)"))

take at stagemetrics.scala:207

4 / 4

take at stagemetrics.scala:207

1 / 1

parquetCasesPopulation: Dataset[Row] = [country: string, avg(infection Per Population): double]

In [82]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    parquetCasesPopulation
    .collect()
)

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd81.sc:2

863 / 863

collect at cmd81.sc:2

...

4 / 4

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 4
numTasks => 1264
elapsedTime => 124218 (2.1 min)
stageDuration => 124050 (2.1 min)
executorRunTime => 728965 (12 min)
executorCpuTime => 95532 (1.6 min)
executorDeserializeTime => 10845 (11 s)
executorDeserializeCpuTime => 7126 (7 s)
resultSerializationTime => 73 (73 ms)
jvmGCTime => 9196 (9 s)
shuffleFetchWaitTime => 667 (0.7 s)
shuffleWriteTime => 16817 (17 s)
resultSize => 2724244 (2.0 MB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 2043142144
recordsRead => 62135
bytesRead => 124565546 (118.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 33032
shuffleTotalBlocksFetched => 31664
shuffleLocalBlocksFetched => 31664
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 2623568 (2.0 MB)
shuffleLocalBytesRead => 2623568 (2.0 MB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk =

res81: Array[Row] = Array(
  [Andorra,3.4400295679401613E-4],
  [Montenegro,2.424873751564056E-4],
  [Luxembourg,1.916931079174976E-4],
  [Aruba,1.769864042262208E-4],
  [Slovenia,1.6312438537043796E-4],
  [Panama,1.5933045595717773E-4],
  [Guam,1.5512777072857723E-4],
  [Belgium,1.4977394087242458E-4],
  [Bahrain,1.4927852711388827E-4],
  [Armenia,1.4709358360766116E-4],
  [Liechtenstein,1.4225295036283427E-4],
  [Qatar,1.410043921276158E-4],
  [Georgia,1.380744125786055E-4],
  [Croatia,1.2360663598057873E-4],
  [Switzerland,1.227607207171266E-4],
  [Portugal,1.192313926859124E-4],
  [Israel,1.1883296974895608E-4],
  [Gibraltar,1.1818342727541574E-4],
  [Argentina,1.1694609939486554E-4],
  [Moldova,1.1205780134087441E-4],
  [Serbia,1.0898339588681602E-4],
  [Spain,1.0604965444992503E-4],
  [Peru,1.0568198964713381E-4],
  [Poland,1.0568017670880612E-4],
  [Chile,1.0481978256189323E-4],
  [France,1.0399872883500103E-4],
  [Hungary,1.0351532863315174E-4],
  [Lithuania,1.0262212428067258E

Porcentaje diario de infecciones

In [83]:
val parquetDailyCasesRate =
parqDF.join(dfPopulation, $"country" === $"countriesAndTerritories")
                .select($"country",
                $"dateRep",
                $"day",
                $"month",
                $"cases",
                $"Population",
                $"cases" / $"Population" as "infection Per Population")
        .orderBy($"dateRep".asc)

parquetDailyCasesRate: Dataset[Row] = [country: string, dateRep: string ... 5 more fields]

In [84]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    parquetDailyCasesRate
    .collect()
)

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd83.sc:2

863 / 863

collect at cmd83.sc:2

...

5 / 5

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 5
numTasks => 1951
elapsedTime => 264979 (4.4 min)
stageDuration => 264663 (4.4 min)
executorRunTime => 1564622 (26 min)
executorCpuTime => 194135 (3.2 min)
executorDeserializeTime => 12285 (12 s)
executorDeserializeCpuTime => 8107 (8 s)
resultSerializationTime => 165 (0.2 s)
jvmGCTime => 24976 (25 s)
shuffleFetchWaitTime => 719 (0.7 s)
shuffleWriteTime => 27843 (28 s)
resultSize => 4603493 (4.0 MB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 1816911872
recordsRead => 150622
bytesRead => 401237892 (382.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 46646
shuffleTotalBlocksFetched => 29184
shuffleLocalBlocksFetched => 29184
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 4105201 (3.0 MB)
shuffleLocalBytesRead => 4105201 (3.0 MB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadTo

res83: Array[Row] = Array(
  [Cambodia,01/01/2020,1,1,0,16758448,0.0],
  [Taiwan,01/01/2020,1,1,0,23824369,0.0],
  [Vietnam,01/01/2020,1,1,0,97490013,0.0],
  [Monaco,01/01/2020,1,1,0,39290,0.0],
  [Norway,01/01/2020,1,1,0,5428594,0.0],
  [Oman,01/01/2020,1,1,0,5128058,0.0],
  [Thailand,01/01/2020,1,1,0,69830779,0.0],
  [Spain,01/01/2020,1,1,0,46757980,0.0],
  [Iceland,01/01/2020,1,1,0,341628,0.0],
  [Afghanistan,01/01/2020,1,1,0,39074280,0.0],
  [Luxembourg,01/01/2020,1,1,0,627704,0.0],
  [Switzerland,01/01/2020,1,1,0,8665615,0.0],
  [Nepal,01/01/2020,1,1,0,29225196,0.0],
  [Georgia,01/01/2020,1,1,0,3987805,0.0],
  [Denmark,01/01/2020,1,1,0,5795780,0.0],
  [Estonia,01/01/2020,1,1,0,1326693,0.0],
  [France,01/01/2020,1,1,0,65298930,0.0],
  [Pakistan,01/01/2020,1,1,0,221612785,0.0],
  [Croatia,01/01/2020,1,1,0,4100719,0.0],
  [Finland,01/01/2020,1,1,0,5542237,0.0],
  [Bahrain,01/01/2020,1,1,0,1711057,0.0],
  [Belarus,01/01/2020,1,1,0,9448772,0.0],
  [Philippines,01/01/2020,1,1,0,10983032

# Visualización de datos con plotly

In [85]:
val (x, y) = Seq(
  "Banana" -> 10,
  "Apple" -> 8,
  "Grapefruit" -> 5
).unzip

Bar(x, y).plot()

x: Seq[String] = List("Banana", "Apple", "Grapefruit")
y: Seq[Int] = List(10, 8, 5)
res84_1: String = "plot-89d368e3-8142-43a2-a27e-c91cd2629195"

## media de infecciones diarias

In [86]:
val (x,y) = infAvgOrDf.collect.map(r=>(r(0).toString, r(1).toString.toDouble)).toList.unzip
Bar(x, y).plot()

collect at cmd85.sc:1

200 / 200

x: List[String] = List(
  "United_States_of_America",
  "India",
  "Brazil",
  "Russia",
  "France",
  "Argentina",
  "United_Kingdom",
  "Italy",
  "Colombia",
  "Spain",
  "Poland",
  "Germany",
  "Turkey",
  "Mexico",
  "Peru",
  "Ukraine",
  "Iran",
  "South_Africa",
  "Chile",
  "Indonesia",
  "Bangladesh",
  "Netherlands",
  "Belgium",
  "Czechia",
  "Iraq",
  "Romania",
  "Morocco",
  "Canada",
  "Philippines",
  "Pakistan",
  "Saudi_Arabia",
  "Portugal",
  "Switzerland",
  "Israel",
  "Hungary",
  "Serbia",
  "Jordan",
  "Sweden",
...
y: List[Double] = List(
  46447.868571428575,
  28321.203438395416,
  19719.862857142856,
  7582.651428571428,
  6791.005714285714,
  5293.851590106007,
  5284.0085714285715,
  5267.748571428571,
  5110.301075268817,
  4958.667621776504,
  3998.8591549295775,
  3820.2228571428573,
  3606.7789855072465,
  3571.554285714286,
  3492.8120567375886,
  3263.282608695652,
  3166.482857142857,
  3053.063829787234,
  2006.7333333333333,
  1801.22448979591

## media de infecciones por km2

In [87]:
val (x,y) = meanInfectionsperKM2DF.collect.map(r=>(r(0).toString, r(1).toString.toFloat)).toList.unzip
Bar(x, y).plot()

collect at cmd86.sc:1

157 / 157

x: List[String] = List(
  "Monaco",
  "Gibraltar",
  "Bahrain",
  "Singapore",
  "Maldives",
  "Malta",
  "Aruba",
  "Belgium",
  "Andorra",
  "Netherlands",
  "Guam",
  "Israel",
  "Luxembourg",
  "Lebanon",
  "Qatar",
  "Liechtenstein",
  "Bermuda",
  "Cura\u00e7ao",
  "Switzerland",
  "Kuwait",
  "Italy",
  "Slovenia",
  "Armenia",
  "Moldova",
  "Bangladesh",
  "Portugal",
  "Poland",
  "France",
  "Montenegro",
  "Austria",
  "Hungary",
  "Germany",
  "Serbia",
  "Jordan",
  "Spain",
  "Slovakia",
  "India",
  "Panama",
...
y: List[Float] = List(
  1.9763314F,
  0.39814815F,
  0.3360843F,
  0.23804082F,
  0.15914285F,
  0.12258171F,
  0.10505618F,
  0.05736941F,
  0.056567993F,
  0.051918827F,
  0.048558153F,
  0.047656838F,
  0.04645812F,
  0.04139474F,
  0.035090588F,
  0.03391577F,
  0.031925924F,
  0.03068482F,
  0.026920669F,
  0.023646325F,
  0.017908985F,
  0.016838819F,
  0.015314984F,
  0.013754947F,
  0.013703283F,
  0.013267124F,
  0.013058352F,
  0.012402372F,
  0.0113

## media de infecciones por densidad de población

In [88]:
val (x,y) = infectionsPerPopulation.collect.map(r=>(r(0).toString, r(1).toString)).toList.unzip
Bar(x, y).plot()

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd87.sc:1

2 / 2

collect at cmd87.sc:1

200 / 200

collect at cmd87.sc:1

200 / 200

x: List[String] = List(
  "Andorra",
  "Montenegro",
  "Luxembourg",
  "Aruba",
  "Slovenia",
  "Panama",
  "Guam",
  "Belgium",
  "Bahrain",
  "Armenia",
  "Liechtenstein",
  "Qatar",
  "Georgia",
  "Croatia",
  "Switzerland",
  "Portugal",
  "Israel",
  "Gibraltar",
  "Argentina",
  "Moldova",
  "Serbia",
  "Spain",
  "Peru",
  "Poland",
  "Chile",
  "France",
  "Hungary",
  "Lithuania",
  "Netherlands",
  "Austria",
  "Colombia",
  "Kuwait",
  "Brazil",
  "Bulgaria",
  "Jordan",
  "Sweden",
  "Malta",
  "Maldives",
...
y: List[String] = List(
  "3.4400295679401656E-4",
  "2.4248737515640602E-4",
  "1.9169310791749766E-4",
  "1.769864042262208E-4",
  "1.6312438537043818E-4",
  "1.5933045595717767E-4",
  "1.5512777072857744E-4",
  "1.4977394087242467E-4",
  "1.4927852711388843E-4",
  "1.470935836076611E-4",
  "1.4225295036283446E-4",
  "1.410043921276158E-4",
  "1.3807441257860555E-4",
  "1.2360663598057857E-4",
  "1.2276072071712674E-4",
  "1.1923139268591238E-4",
  "1.18832969748956

## porcentaje diario de infectados

In [89]:
val (x,y) = diaryInfectionsDF.filter($"country" === "Spain").collect.map(r=>(r(1).toString, r(6).toString)).toList.unzip
Bar(x, y).plot()


collect at cmd87.sc:1

157 / 157

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd88.sc:1

...

collect at cmd88.sc:1

2 / 2

collect at cmd88.sc:1

200 / 200

x: List[String] = List(
  "2019-12-31",
  "2020-01-01",
  "2020-01-02",
  "2020-01-03",
  "2020-01-04",
  "2020-01-05",
  "2020-01-06",
  "2020-01-07",
  "2020-01-08",
  "2020-01-09",
  "2020-01-10",
  "2020-01-11",
  "2020-01-12",
  "2020-01-13",
  "2020-01-14",
  "2020-01-15",
  "2020-01-16",
  "2020-01-17",
  "2020-01-18",
  "2020-01-19",
  "2020-01-20",
  "2020-01-21",
  "2020-01-22",
  "2020-01-23",
  "2020-01-24",
  "2020-01-25",
  "2020-01-26",
  "2020-01-27",
  "2020-01-28",
  "2020-01-29",
  "2020-01-30",
  "2020-01-31",
  "2020-02-01",
  "2020-02-02",
  "2020-02-03",
  "2020-02-04",
  "2020-02-05",
  "2020-02-06",
...
y: List[String] = List(
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "0.0",
  "2.138672372074243E-8",
  "0.0",
  "0.0",
  "0.0",

## comparacion entre paises de crecimiento de la enfermedad

In [90]:
val y = diaryInfectionsDF.filter($"country" === "Spain").select($"infection Per Population").
    collect.map(r => r(0).toString.toDouble).toList

val x = diaryInfectionsDF.filter($"country" === "Spain").select($"to_date").collect.toList.map(_.toString)

val y1 = diaryInfectionsDF.filter($"country" === "Italy").select($"infection Per Population").
    collect.map(r => r(0).toString.toDouble).toList
val x1 = diaryInfectionsDF.filter($"country" === "Italy").select($"to_date").collect.toList.map(_.toString)

val data = Seq(
    Scatter(x,y).withName("Spain"),
    Scatter(x1,y1,mode = ScatterMode(ScatterMode.Lines),
  line = Line(color = Color.StringColor("#7F7F7F"))).withName("Italy")
).map(_.withFill(Fill.ToNextY).withStackgroup("A"))

plot(data)

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd89.sc:1

2 / 2

collect at cmd89.sc:1

...

collect at cmd89.sc:8

2 / 2

collect at cmd89.sc:8

200 / 200

y: List[Double] = List(
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.138672372074243E-8,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
...
x: List[String] = List(
  "[2019-12-31]",
  "[2020-01-01]",
  "[2020-01-02]",
  "[2020-01-03]",
  "[2020-01-04]",
  "[2020-01-05]",
  "[2020-01-06]",
  "[2020-01-07]",
  "[2020-01-08]",
  "[2020-01-09]",
  "[2020-01-10]",
  "[2020-01-11]",
  "[2020-01-12]",
  "[2020-01-13]",
  "[2020-01-14]",
  "[2020-01-15]",
  "[2020-01-16]",
  "[2020-01-17]",
  "[2020-01-18]",
  "[2020-01-19]",
  "[2020-01-20]",
  "[2020-01-21]",
  "[2020-01-22]",
  "[2020-01-23]",
  "[2020-01-24]",
  "[2020-01-25]",
  "[2020-01-26]",
  "[2020-01-27]",
  "[2020-01-28]",
  "[2020-01-29]",
  "[2020-01-30]",
  "[2020-01-31]",
  "[2020-02-01]",
  "[2020-02-02]",
  "[2020-02-03]",
  "[2020-02-04]",
  "[2020-02-05]",

## porcentaje de la población vacunada

## crecimiento de la vacunacion con respecto a la población

In [95]:
val y = megaQuerie.filter($"country" === "Chile").select($"vaccination Per Population" * 10000000).
    collect.map(r => r(0).toString.toDouble).toList

val x = megaQuerie.filter($"country" === "Chile").select($"date").collect.toList.map(_.toString)

val y1 = megaQuerie.filter($"country" === "Chile").select($"people_vaccinated").
    collect.map(r => r(0).toString.toDouble).toList
val x1 = megaQuerie.filter($"country" === "Chile").select($"date").collect.toList.map(_.toString)

val data = Seq(
    Scatter(x,y).withName("% population"),
    Scatter(x1,y1).withName("Vaccines administrated")
).map(_.withFill(Fill.ToNextY).withStackgroup("A"))

val myLayout =
  Layout()
    .withTitle("CHILE")

plot(data,myLayout)

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd94.sc:1

...

collect at cmd94.sc:8

5 / 5

collect at cmd94.sc:8

131 / 131

y: List[Double] = List(
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  219.4,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  2715.1,
  4355.3,
  4355.3,
  4355.3,
  4355.3,
...
x: List[String] = List(
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-24]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-12-25]",
  "[2020-

# Visualización de eficiencia

para la querie de media de infecciones diarias:

In [96]:
val (x, y) = Seq(
    "RDD" -> runWithOutput(infectionAvgRDD.collect),
    "DataSet" -> runWithOutput(avgDS.collect),
    "DataFrame" -> runWithOutput(infAvgOrDf.collect)
).unzip

Bar(x, y).plot()

collect at cmd95.sc:2

2 / 2

Took 40


collect at cmd95.sc:3

200 / 200

Took 460
Took 466


x: Seq[String] = List("RDD", "DataSet", "DataFrame")
y: Seq[Int] = List(40, 460, 466)
res95_1: String = "plot-135d63e7-2db3-4bd7-aad0-3dae884162d1"

para la querie de infecciones por km2

In [98]:
val (x, y) = Seq(
    "Not Optimized RDD" -> runWithOutput(notOptimizedRDD.collect),
    "RDD" -> runWithOutput(meanInfectionsRDD.collect),
    "DataSet" -> runWithOutput(meanInfectionsperKM2DS.collect),
    "DataFrame" -> runWithOutput(meanInfectionsperKM2DF.collect),
    "DataFrame using Parquet" -> runWithOutput(parquetCasesKM2.collect)
).unzip

Bar(x, y).plot()

map at cmd39.sc:1

2 / 2

map at cmd39.sc:5

2 / 2

collect at cmd97.sc:2

...

Took 354
Took 365


collect at cmd97.sc:5

157 / 157

Took 355


x: Seq[String] = List(
  "Not Optimized RDD",
  "RDD",
  "DataSet",
  "DataFrame",
  "DataFrame using Parquet"
)
y: Seq[Int] = List(1683, 256, 354, 365, 355)
res97_1: String = "plot-61f91fed-1c15-4685-97d2-c869fb5a1eaf"

para la querie de infecciones por número de habitantes

In [100]:
val (x, y) = Seq(
    "DataSet" -> runWithOutput(meanInfectionPerPopulationDS.collect),
    "DataFrame" -> runWithOutput(infectionsPerPopulation.collect),
    "DataFrame vaccinations" -> runWithOutput(megaQuerie.collect),
    "DataFrame using Parquet" -> runWithOutput(parquetCasesPopulation.collect)
).unzip

Bar(x, y).plot()

collect at cmd99.sc:2

157 / 157

Took 380


collect at cmd99.sc:3

157 / 157

Took 649
Took 468


collect at cmd99.sc:4

129 / 129

Took 472


x: Seq[String] = List(
  "DataSet",
  "DataFrame",
  "DataFrame vaccinations",
  "DataFrame using Parquet"
)
y: Seq[Int] = List(380, 649, 468, 472)
res99_1: String = "plot-2d0bb00c-f16f-4846-98df-0dba95affc43"